This code features finite volume implimentation of electrolyte diffusion of lithium. This notebook details FVM calculations and its Julia implimentation. The notebook helps generate gradient,divergence, mean calculation matrices to be incorporated into any electrochemical model. The notebook also features a simple example of how to use tthese matrices to solve for concentration profile  and effect of mesh size on the solution.

The code is implimented by Jishnu Ayyangatu Kuzhiyil. At the time of writing the code, Jishnu is a PhD student at the University of Warwick.

In [127]:
# Getting into implimentation, I will be using the Julia programming language. I will be using the following packages:

using LinearAlgebra,DifferentialEquations,Plots,SparseArrays,BenchmarkTools,PreallocationTools

#Chen2020 parameters
Ln=85.2*1e-6  #thickness of negative elctrode
Ls=12 *1e-6   #thickness of separator
Lp=75.6*1e-6  #thickness of positive electrode

Nn=15 #number of FVM CVs in negative electrode
Ns=6 #number of FVM CVs in separator
Np=15 #number of FVM CVs in positive electrode

I=5 #current in A
F=96485 #Faraday's constant
A=0.1 #area of electrode in m^2
t⁺=0.2594 #transference number 

0.2594

Lets calculate the gradient of electrolyte concentration at interface between control volumes. The gradient can be calculated as right value minus left value divided by the distance between the two control volumes. 

In [128]:

function electrolyte_matrices(Nn,Ns,Np,Ln,Ls,Lp; save_matrices=false)
   
   """ # This function will return the matrices required for solving the electrolyte concentration equation. The matrices are:
    # 1. Gradient matrix
    # 2. Divergence matrix
    # 3. Mean matrix
    # 4. Source term vector
    
    # The gradient matrix is a Nx(N-1) matrix, where N is the total number of CVs. The gradient matrix is used to calculate the gradient of the electrolyte concentration at the interfaces of the CVs.
    # The divergence matrix is a Nx(N-1) matrix, where N is the total number of CVs. The divergence matrix is used to calculate the divergence of the electrolyte flux at the interfaces of the CVs.
    # The mean matrix is a NxN matrix, where N is the total number of CVs. The mean matrix is used to calculate the average of the electrolyte concentration at the interfaces of the CVs.
    # The source term vector is a Nx1 vector, where N is the total number of CVs. The source term vector is used to calculate the source term in the electrolyte concentration equation.
    
    # The inputs to the function are:
    # 1. Nn: Number of CVs in the negative electrode
    # 2. Ns: Number of CVs in the separator
    # 3. Np: Number of CVs in the positive electrode
    # 4. Ln: Thickness of negative electrode
    # 5. Ls: Thickness of separator
    # 6. Lp: Thickness of positive electrode
    
    # The outputs of the function are:
    # 1. grad_interface_electrolyte: Gradient matrix
    # 2. div_interface_electrolyte: Divergence matrix
    # 3. mean_interface_electrolyte: Mean matrix
    # 4. source: Source term vector
    
    # The function is called as follows:
    # grad_interface_electrolyte,div_interface_electrolyte,mean_interface_electrolyte,source = electrolyte_matrices(Nn,Ns,Np,Ln,Ls,Lp)"""
    
    # First we will calculate the length of each CV
    N=Nn+Ns+Np
    Δx_n = Ln/Nn
    Δx_s = Ls/Ns
    Δx_p = Lp/Np

    Δx_ns = (Δx_n + Δx_s)/2
    Δx_sp= (Δx_s + Δx_p)/2

    # To impliment gradient calculation as a matrix multiplication, we need to define the following matrices

    e_vec=[1/Δx_n.*ones(Nn-1); 1/Δx_ns; 1/Δx_s.*ones(Ns-1); 1/Δx_sp; 1/Δx_p.*ones(Np-1)]
    grad_interface_electrolyte = spdiagm(N-1,N,0=>-e_vec,1=>e_vec)
    # The grad electrolyte matrix is a sparse matrix, when multiplied by the electrolyte concentration vector, it will return the gradient of the electrolyte concentration vector
    # at all the interfaces between the CVs. The gradient of the electrolyte concentration at both boundaries are zero based on no flux at boundary.


    #Now lets calculate a matrix which multiplied with a parameter at centre of CVs will return the average of the parameter at the interfaces of the CVs
    left_weight_ns = Δx_n/(Δx_n+Δx_s)
    right_weight_ns = Δx_s/(Δx_n+Δx_s)

    left_weight_sp = Δx_s/(Δx_s+Δx_p)
    right_weight_sp = Δx_p/(Δx_s+Δx_p)

    left_weight=[0.5*ones(Nn-1);left_weight_ns;0.5*ones(Ns-1);left_weight_sp;0.5*ones(Np-1)]
    right_weight=[0.5*ones(Nn-1);right_weight_ns;0.5*ones(Ns-1);right_weight_sp;0.5*ones(Np-1)]
    mean_interface_electrolyte = spdiagm(N-1,N,0=>left_weight,1=>right_weight)
    # The mean electrolyte matrix is a sparse matrix, when multiplied by the electrolyte concentration vector, it will return the average of the electrolyte concentration vector
    #This can be used to calculate the average bruggman and porosity at interfaces.

    #Now lets calculate the divergence matrix. In cartician coordinates, divergence is essentially the gradient of flux at ineterfaces. Ideally, we will have N+1 interfaces, and divergence 
    # matrix will be a (N+1)xN matrix. However, we have zero flux at both boundaries , so we will have N-1 interfaces to consider and the matrix now is a Nx(N-1) matrix.
    left = [1/Δx_n.*ones(Nn-1); 1/Δx_s.*ones(Ns); 1/Δx_p.*ones(Np)]
    right = [1/Δx_n.*ones(Nn); 1/Δx_s.*ones(Ns); 1/Δx_p.*ones(Np-1)]
    div_interface_electrolyte = spdiagm(N,N-1,0=>right,-1=>-left)
    # The div electrolyte matrix is a sparse matrix, when multiplied by the electrolyte flux vector, it will return the gradient of the electrolyte flux vector

    #Now lets calculate the source term at each CV
    Source_n = (1-t⁺)/(F*A*Ln)
    Source_s = 0
    Source_p = -(1-t⁺)/(F*A*Lp)
    source = [Source_n*ones(Nn);Source_s*ones(Ns);Source_p*ones(Np)]


    if save_matrices
        open("Eelectrolyte_matrices.txt", "w") do f
            write(f, "Mesh_neg_sep_pos: $([Nn,Ns,Np])\n")
            write(f, "grad_interface_electrolyte = $grad_interface_electrolyte\n")
            write(f, "div_interface_electrolyte = $div_interface_electrolyte\n")
            write(f, "mean_interface_electrolyte = $mean_interface_electrolyte\n")
            write(f, "source = $source\n")

            write(f,"grad_ce = grad_interface_electrolyte * ce_vec \n")
            write(f,"mean_brug=mean_interface_electrolyte*brugg_vec\n")
            write(f,"flux_at_interfaces= D.* brugg_vec .* grad_ce \n")
            write(f,"dc_dt = div_interface_electrolyte * ce_vec (just because of diffusion) \n")
            write(f,"dc_dt=dc_dt .+ source*I \n")

        end 
    end

    return grad_interface_electrolyte,div_interface_electrolyte,mean_interface_electrolyte,source

end 

electrolyte_matrices(Nn,Ns,Np,Ln,Ls,Lp; save_matrices=true)

(sparse([1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  31, 31, 32, 32, 33, 33, 34, 34, 35, 35], [1, 2, 2, 3, 3, 4, 4, 5, 5, 6  …  31, 32, 32, 33, 33, 34, 34, 35, 35, 36], [-176056.33802816903, 176056.33802816903, -176056.33802816903, 176056.33802816903, -176056.33802816903, 176056.33802816903, -176056.33802816903, 176056.33802816903, -176056.33802816903, 176056.33802816903  …  -198412.6984126984, 198412.6984126984, -198412.6984126984, 198412.6984126984, -198412.6984126984, 198412.6984126984, -198412.6984126984, 198412.6984126984, -198412.6984126984, 198412.6984126984], 35, 36), sparse([1, 2, 2, 3, 3, 4, 4, 5, 5, 6  …  31, 32, 32, 33, 33, 34, 34, 35, 35, 36], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  31, 31, 32, 32, 33, 33, 34, 34, 35, 35], [176056.33802816903, -176056.33802816903, 176056.33802816903, -176056.33802816903, 176056.33802816903, -176056.33802816903, 176056.33802816903, -176056.33802816903, 176056.33802816903, -176056.33802816903  …  198412.6984126984, -198412.6984126984, 198412.6984126984, -198

Now lets test it out for various mesh sizes and see how the gradient changes.


In [138]:
Test_mesh_sizes=[(6,3,6),(9,3,9),(30,15,30)]
time_to_plot=100.0
duration=1800.0
plots=plot()

ϵ_n=0.25
ϵ_s=0.47
ϵ_p=0.335


for (Nn,Ns,Np) in Test_mesh_sizes

    N=Nn+Ns+Np
    grad_interface_electrolyte,div_interface_electrolyte,mean_interface_electrolyte,source=electrolyte_matrices(Nn,Ns,Np,Ln,Ls,Lp;save_matrices=false)
    ϵ=[ϵ_n*ones(Nn);ϵ_s*ones(Ns);ϵ_p*ones(Np)]
    Brugg=ϵ.^1.5

    function ODE_fctn!(dc,c,p,t)
        
        grad_ce=grad_interface_electrolyte*c
        mean_brug=mean_interface_electrolyte*Brugg
        D_e_node = 8.794e-11 .* (c ./ 1000) .^ 2 .- 3.972e-10 .* (c ./ 1000) .+ 4.862e-10
        D_e_node = mean_interface_electrolyte * D_e_node 
        flux_at_interfaces= D_e_node.* mean_brug .* grad_ce
        dc_dt=div_interface_electrolyte*flux_at_interfaces
        dc_dt=dc_dt.+source*I
        dc .= dc_dt./ϵ
    end

    c0=1e3*ones(N)
    tspan=(0.0,duration)
    prob=ODEProblem(ODE_fctn!,c0,tspan)
    
    
    
    solution=sol(time_to_plot)
    x_points_n=[(i-0.5)*Ln/Nn for i in 1:Nn]
    x_points_s=[Ln+(i-0.5)*Ls/Ns for i in 1:Ns]
    x_points_p=[Ln+Ls+(i-0.5)*Lp/Np for i in 1:Np]
    x_points=[0;x_points_n;x_points_s;x_points_p;Ln+Ls+Lp]

    y_point_0 = solution[1] - 0.25*(solution[2]-solution[1]) #1.5*solution[1]-0.5*solution[2]
    
    y_point_end = solution[end] + 0.25*(solution[end]-solution[end-1]) #
    #1.5*solution[end]-0.5*solution[end-1]
    y_points = [y_point_0;solution;y_point_end]

    plot!(plots,1e6.*x_points,y_points,label="Nn=$Nn, Ns=$Ns, Np=$Np")


end 

plot!(plots,xlabel="x (μm)",ylabel="Electrolyte concentration (mol/m^3)",title="FVM implimentation of 1D electrolyte diffusion",legend=:topright)
plot!(plots,dpi=300,lw=2,markersize=2,framestyle=:box)
annotate!(plots, 100, 1000.0, text("Time = $time_to_plot s", 10, :left))
annotate!(plots, 100, 1100.0, text("Current = $I A", 10, :left))
savefig(plots,"Electrolyte_concentration.png")


  0.450191 seconds (1.12 M allocations: 56.810 MiB, 99.11% compilation time)


  0.005525 seconds (13.82 k allocations: 1.931 MiB)


  0.087947 seconds (42.26 k allocations: 14.821 MiB, 55.48% gc time)


"m:\\Calendar ageing GB\\codes\\Electrolyte_concentration.png"